# Compilation Examples with Routing

In [1]:

from pytket import Circuit
from pytket.extensions.aqt.backends.aqt_multi_zone import AQTMultiZoneBackend
from pytket.extensions.aqt.multi_zone_architecture.circuit_routing.settings import (
    RoutingAlg,
    RoutingSettings,
)
from pytket.extensions.aqt.multi_zone_architecture.compilation_settings import (
    CompilationSettings,
)
from pytket.extensions.aqt.multi_zone_architecture.graph_algs.mt_kahypar import (
    MtKahyparConfig,
    configure_mtkahypar,
)
from pytket.extensions.aqt.multi_zone_architecture.initial_placement.settings import (
    InitialPlacementAlg,
    InitialPlacementSettings,
)
from pytket.extensions.aqt.multi_zone_architecture.named_architectures import (
    four_zones_in_a_line,
    grid12,
    grid12_mod,
    racetrack,
    racetrack_4_gatezones,
)

First lets define some circuits

In [2]:
## A simple GHZ circuit
ghz_circuit = Circuit(16)
ghz_circuit.H(0)
for i in range(ghz_circuit.n_qubits - 1):
    ghz_circuit.CX(i, i + 1)
ghz_circuit.measure_all()

## Quantum advantage circuit template
def quantum_advantage_circuit(Lx: int, Ly: int) -> Circuit:
    N = Lx * Ly
    Ntrot = 30
    dt = 0.2
    hx = 2
    J = 1

    def c(x, y):
        if (x - y) % N == Lx or (y - x) % N == Lx:
            return 1
        if x // Lx == y // Lx and x == y - (y % Lx) + (((y % Lx) + 1) % Lx):
            return 1
        if x // Lx == y // Lx and y == x - (x % Lx) + (((x % Lx) + 1) % Lx):
            return 1
        return 0

    coupling_list = []
    for i in range(N):
        for j in range(i + 1, N):
            if c(i, j) == 1:
                coupling_list.append([i, j])

    U = Circuit(N)

    for t in range(Ntrot):
        n_trotter_steps = t

        if n_trotter_steps == 0:
            for j in range(N):
                U.Rx(-2 * dt * hx / 2, j)

        for coupling in coupling_list:
            U.ZZPhase(-2 * dt * J, coupling[0], coupling[1])
        for j in range(N):
            U.Rx(-2 * dt * hx, j)

        if n_trotter_steps == Ntrot - 1:
            for j in range(N):
                U.Rx(2 * dt * hx / 2, j)
    return U

## 56 qubit quantum advantage circuit
advantage_circuit_56 = quantum_advantage_circuit(7, 8)

## 30 qubit quantum advantage circuit
advantage_circuit_30 = quantum_advantage_circuit(5, 6)

def build_qft_circuit(n_qubits: int) -> Circuit:
    circ = Circuit(n_qubits, name="QFT")
    for i in range(n_qubits):
        circ.H(i)
        for j in range(i + 1, n_qubits):
            circ.CU1(1 / 2 ** (j - i), j, i)
    for k in range(n_qubits // 2):
        circ.SWAP(k, n_qubits - k - 1)
    return circ

## 16 qft circuit
qft_circ = build_qft_circuit(16)



Now some MultiZone Backends

In [3]:
line_backend = AQTMultiZoneBackend(
    architecture=four_zones_in_a_line, access_token="invalid"
)

racetrack_backend = AQTMultiZoneBackend(
    architecture=racetrack, access_token="invalid"
)

racetrack_4_gatezones_backend = AQTMultiZoneBackend(
    architecture=racetrack_4_gatezones, access_token="invalid"
)

grid_backend = AQTMultiZoneBackend(
    architecture=grid12, access_token="invalid"
)

grid_mod_backend = AQTMultiZoneBackend(
    architecture=grid12_mod, access_token="invalid"
)


And some different compilation settings

In [4]:
# This can be used to configure the number of threads used and random seed
# for mt-kahypar. It is not required (then default will be used) and can only
# be set once
configure_mtkahypar(
    MtKahyparConfig(
        n_threads = 1,
        random_seed = 13
    )
)
order_init = InitialPlacementSettings(
    algorithm=InitialPlacementAlg.qubit_order,
    zone_free_space=2,
    max_depth=200,
)
graph_routing = RoutingSettings(
    algorithm=RoutingAlg.graph_partition,
    ignore_swap_costs = False,
    debug_level=0,
)
graph_compilation_settings = CompilationSettings(
    pytket_optimisation_level=1,
    initial_placement=order_init,
    routing=graph_routing,
)

greedy_routing = RoutingSettings(
    algorithm=RoutingAlg.greedy,
    debug_level=0,
)
greedy_compilation_settings = CompilationSettings(
    pytket_optimisation_level=1,
    initial_placement=order_init,
    routing=greedy_routing,
)

## GHZ circuit on a line architecture with two central gate zones and two outer memory zones (8-6-6-8)

In [6]:
ghz_precompiled = line_backend.precompile_circuit(ghz_circuit, graph_compilation_settings)
print("#1qb gates =", ghz_precompiled.n_1qb_gates())
print("#gate depth =", ghz_precompiled.depth())
print("#2qb gates =", ghz_precompiled.n_2qb_gates())
print("#2qb gate depth =", ghz_precompiled.depth_2q())
ghz_graph_routed = line_backend.route_precompiled(ghz_precompiled, graph_compilation_settings)
ghz_greedy_routed = line_backend.route_precompiled(ghz_precompiled, greedy_compilation_settings)
ghz_greedy_routed_legacy = line_backend.route_precompiled(ghz_precompiled, greedy_compilation_settings, use_legacy_routing=True)
print("Shuttles, swaps graph: ", ghz_graph_routed.get_n_shuttles(), ", ", ghz_graph_routed.get_n_pswaps())
print("Shuttles, swaps greedy: ", ghz_greedy_routed.get_n_shuttles(), ", ", ghz_greedy_routed.get_n_pswaps())
print("Shuttles, swaps greedy legacy: ", ghz_greedy_routed_legacy.get_n_shuttles(), ", ", ghz_greedy_routed_legacy.get_n_pswaps())

#1qb gates = 59
#gate depth = 46
#2qb gates = 15
#2qb gate depth = 15
Shuttles, swaps graph:  16 ,  39
Shuttles, swaps greedy:  13 ,  62
Shuttles, swaps greedy legacy:  22 ,  81


## QFT circuit on a line architecture with two central gate zones and two outer memory zones (8-6-6-8)

In [11]:
qft_precompiled = line_backend.precompile_circuit(qft_circ, graph_compilation_settings)
print("#1qb gates =", qft_precompiled.n_1qb_gates())
print("#gate depth =", qft_precompiled.depth())
print("#2qb gates =", qft_precompiled.n_2qb_gates())
print("#2qb gate depth =", qft_precompiled.depth_2q())
qft_graph_routed = line_backend.route_precompiled(qft_precompiled, graph_compilation_settings)
qft_greedy_routed = line_backend.route_precompiled(qft_precompiled, greedy_compilation_settings)
qft_greedy_routed_legacy = line_backend.route_precompiled(qft_precompiled, greedy_compilation_settings, use_legacy_routing=True)
print("Shuttles, swaps graph: ", qft_graph_routed.get_n_shuttles(), ", ", qft_graph_routed.get_n_pswaps())
print("Shuttles, swaps greedy: ", qft_greedy_routed.get_n_shuttles(), ", ", qft_greedy_routed.get_n_pswaps())
print("Shuttles, swaps greedy legacy: ", qft_greedy_routed_legacy.get_n_shuttles(), ", ", qft_greedy_routed_legacy.get_n_pswaps())

#1qb gates = 569
#gate depth = 155
#2qb gates = 264
#2qb gate depth = 61
Shuttles, swaps graph:  94 ,  209
Shuttles, swaps greedy:  89 ,  295
Shuttles, swaps greedy legacy:  312 ,  934


## 56 qubit "quantum advantage" circuit on all gate-zone "racetrack" architecture

In [12]:

adv_precomp = racetrack_backend.precompile_circuit(advantage_circuit_56, graph_compilation_settings)
print("#1qb gates =", adv_precomp.n_1qb_gates())
print("#gate depth =", adv_precomp.depth())
print("#2qb gates =",  adv_precomp.n_2qb_gates())
print("#2qb gate depth =", adv_precomp.depth_2q())
adv_graph_routed = racetrack_backend.route_precompiled(adv_precomp, graph_compilation_settings)
adv_greedy_routed = racetrack_backend.route_precompiled(adv_precomp, greedy_compilation_settings)
adv_greedy_routed_legacy = racetrack_backend.route_precompiled(adv_precomp, greedy_compilation_settings, use_legacy_routing=True)
print("Shuttles, swaps graph: " , adv_graph_routed.get_n_shuttles(),", ", adv_graph_routed.get_n_pswaps() )
print("Shuttles, swaps greedy: ", adv_greedy_routed.get_n_shuttles(),", ", adv_greedy_routed.get_n_pswaps())
print("Shuttles, swaps greedy legacy: ", adv_greedy_routed_legacy.get_n_shuttles(),", ", adv_greedy_routed_legacy.get_n_pswaps())

#1qb gates = 8700
#gate depth = 2162
#2qb gates = 6720
#2qb gate depth = 1094
Shuttles, swaps graph:  17130 ,  38924
Shuttles, swaps greedy:  16945 ,  47568
Shuttles, swaps greedy legacy:  16103 ,  56830


## 56 qubit "quantum advantage" circuit on 4 sequential gate-zone "racetrack" architecture

In [13]:

adv_precomp2 = racetrack_4_gatezones_backend.precompile_circuit(advantage_circuit_56, graph_compilation_settings)
print("#1qb gates =", adv_precomp2.n_1qb_gates())
print("#gate depth =", adv_precomp2.depth())
print("#2qb gates =", adv_precomp2.n_2qb_gates())
print("#2qb gate depth =", adv_precomp2.depth_2q())
adv_graph_routed2 = racetrack_4_gatezones_backend.route_precompiled(adv_precomp2, graph_compilation_settings)
adv_greedy_routed2 = racetrack_4_gatezones_backend.route_precompiled(adv_precomp2, greedy_compilation_settings)
adv_greedy_routed2_legacy = racetrack_4_gatezones_backend.route_precompiled(adv_precomp2, greedy_compilation_settings, use_legacy_routing=True)
print("Shuttles, swaps graph: " , adv_graph_routed2.get_n_shuttles() , ",", adv_graph_routed2.get_n_pswaps())
print("Shuttles, swaps greedy: ", adv_greedy_routed2.get_n_shuttles(), ",", adv_greedy_routed2.get_n_pswaps())
print("Shuttles, swaps greedy legacy: ", adv_greedy_routed2_legacy.get_n_shuttles(), ",", adv_greedy_routed2_legacy.get_n_pswaps())

#1qb gates = 8700
#gate depth = 2162
#2qb gates = 6720
#2qb gate depth = 1094
Shuttles, swaps graph:  12164 , 44982
Shuttles, swaps greedy:  15036 , 63412
Shuttles, swaps greedy legacy:  18826 , 85168


## Grid Architecture: All gate zones

In [14]:

adv_precomp = grid_backend.precompile_circuit(advantage_circuit_30, graph_compilation_settings)
print("#1qb gates =", adv_precomp.n_1qb_gates())
print("#gate depth =", adv_precomp.depth())
print("#2qb gates =", adv_precomp.n_2qb_gates())
print("#2qb gate depth =", adv_precomp.depth_2q())
adv_graph_routed = grid_backend.route_precompiled(adv_precomp, graph_compilation_settings)
adv_greedy_routed = grid_backend.route_precompiled(adv_precomp, greedy_compilation_settings)
adv_greedy_routed_legacy = grid_backend.route_precompiled(adv_precomp, greedy_compilation_settings, use_legacy_routing=True)
print("Shuttles, swaps graph: " , adv_graph_routed.get_n_shuttles() , ",", adv_graph_routed.get_n_pswaps())
print("Shuttles, swaps greedy: ", adv_greedy_routed.get_n_shuttles(), ",", adv_greedy_routed.get_n_pswaps())
print("Shuttles, swaps greedy legacy: ", adv_greedy_routed_legacy.get_n_shuttles(), ",", adv_greedy_routed_legacy.get_n_pswaps())

#1qb gates = 4654
#gate depth = 1269
#2qb gates = 3600
#2qb gate depth = 640
Shuttles, swaps graph:  2815 , 4160
Shuttles, swaps greedy:  2385 , 4131
Shuttles, swaps greedy legacy:  2550 , 10156


## Grid Architecture: Four gate zones

In [16]:
adv_gm_precompiled = grid_mod_backend.precompile_circuit(advantage_circuit_30, graph_compilation_settings)
print("#1qb gates =", adv_gm_precompiled.n_1qb_gates())
print("#gate depth =", adv_gm_precompiled.depth())
print("#2qb gates =", adv_gm_precompiled.n_2qb_gates())
print("#2qb gate depth =", adv_gm_precompiled.depth_2q())
adv_gm_graph_routed = grid_mod_backend.route_precompiled(adv_gm_precompiled, graph_compilation_settings)
adv_gm_greedy_routed = grid_mod_backend.route_precompiled(adv_gm_precompiled, greedy_compilation_settings)
adv_gm_greedy_routed_legacy = grid_mod_backend.route_precompiled(adv_gm_precompiled, greedy_compilation_settings, use_legacy_routing=True)
print("Shuttles, swaps graph: " , adv_gm_graph_routed.get_n_shuttles() , ",", adv_gm_graph_routed.get_n_pswaps() )
print("Shuttles, swaps greedy: ", adv_gm_greedy_routed.get_n_shuttles(), ",", adv_gm_greedy_routed.get_n_pswaps())
print("Shuttles, swaps greedy legacy: ", adv_gm_greedy_routed_legacy.get_n_shuttles(), ",", adv_gm_greedy_routed_legacy.get_n_pswaps())

#1qb gates = 4654
#gate depth = 1269
#2qb gates = 3600
#2qb gate depth = 640
Shuttles, swaps graph:  3547 , 4118
Shuttles, swaps greedy:  2359 , 2743
Shuttles, swaps greedy legacy:  3633 , 6514
